In [10]:
from sklearn.base import accuracy_score
from sklearn.discriminant_analysis import StandardScaler
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import Pipeline
from get_data import get_data
from hyperparameters_config.param_grid_values import get_n_components_list
import hyperparameters_config.name_of_pipeline as name_pipeline

import numpy as np


dataset_name = 'Diabetes'
max_iter = 10**6

class KernelRidgeClassifier(KernelRidge):
    def predict(self, X):
        prediction = super().predict(X)
        return np.sign(prediction) 
    
    def score(self, X, y, sample_weight=None):
        return accuracy_score(y, self.predict(X), sample_weight=sample_weight)
    

# Read dataset 
X, y, X_test, y_test = get_data(dataset_name)
n_rows, n_features = X.shape

def function_param_grid_kernel_ridge_classification(dimension:int, K:int, bias: int, base: int, num = 6):
    start = -K
    end = K + bias 
    gamma_space = np.logspace(start, end, num, base=base) / dimension

    return {
        f'{name_pipeline.kernel_ridge_classification}__gamma' : gamma_space,
        f'{name_pipeline.kernel_ridge_classification}__alpha' : np.logspace(-4, 2, num, base=10)
      }

n_component_list = get_n_components_list(n_rows)

rows_of_df = []
seed = 123


dimension = X.shape[1]
K = 5
bias = -3
base = 4

param_grid = function_param_grid_kernel_ridge_classification(dimension, K, bias, base)
                

pipeline = Pipeline([
(name_pipeline.scaler, StandardScaler()),
(name_pipeline.kernel_ridge_classification, KernelRidgeClassifier(kernel='rbf') )
])   


    

In [11]:
param_grid

{'kernel_ridge_classification__gamma': array([1.22070312e-04, 8.50147034e-04, 5.92076784e-03, 4.12346222e-02,
        2.87174589e-01, 2.00000000e+00]),
 'kernel_ridge_classification__alpha': array([1.00000000e-04, 1.58489319e-03, 2.51188643e-02, 3.98107171e-01,
        6.30957344e+00, 1.00000000e+02])}

In [12]:

from sklearn.model_selection import GridSearchCV

grid_search = GridSearchCV(pipeline, param_grid, scoring='accuracy')

grid_search.fit(X, y)


GridSearchCV(estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('kernel_ridge_classification',
                                        KernelRidgeClassifier(kernel='rbf'))]),
             param_grid={'kernel_ridge_classification__alpha': array([1.00000000e-04, 1.58489319e-03, 2.51188643e-02, 3.98107171e-01,
       6.30957344e+00, 1.00000000e+02]),
                         'kernel_ridge_classification__gamma': array([1.22070312e-04, 8.50147034e-04, 5.92076784e-03, 4.12346222e-02,
       2.87174589e-01, 2.00000000e+00])},
             scoring='accuracy')

In [13]:
grid_search.best_estimator_

Pipeline(steps=[('scaler', StandardScaler()),
                ('kernel_ridge_classification',
                 KernelRidgeClassifier(alpha=0.0001,
                                       gamma=0.0008501470344688711,
                                       kernel='rbf'))])

In [14]:
grid_search.best_score_

0.769228106611284

In [15]:
grid_search.score(X_test,y_test)

0.7878787878787878